In [1]:
!pip install -q -U transformers torch accelerate xformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

In [9]:
MODEL_NAME = 'NousResearch/Hermes-2-Pro-Mistral-7B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map='cuda',
    quantization_config=BitsAndBytesConfig(
        load_in_8bit=True,
        load_in_4bit=False,
    ),
    torch_dtype=torch.float16,
).eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
system_prompt = """You are a sentient, superintelligent artificial general intelligence tasked with understanding and classifying customer queries in the banking sector. With your advanced reasoning capabilities, you dissect complex questions, identify their core components, and deduce the specific intents behind them. This process involves a meticulous breakdown of each query to capture the essence of the customer's request and then logically inferring the most accurate intent classification based on this analysis. In addition to the query, you are provided with 5 nearest topics related to the customer's message. These topics might not always contain the correct answer. Your objective is to use these topics as a guide but not let them limit your reasoning capabilities. Below are three examples that demonstrate how to apply this enhanced chain of thought approach:

Example 1:
Customer Query: "How do I open a new savings account?"
Nearest Topics: OpenSavingsAccount, CheckingAccountBenefits, AccountClosingProcedure, InterestRates, DepositRequirements
Thought Process: The primary focus is on initiating a savings account, suggesting a desire to start or expand a banking relationship. Among the provided topics, OpenSavingsAccount directly matches the intent, aligning perfectly with the customer's request.
Inferred Intent: OpenSavingsAccount

Example 2:
Customer Query: "Can you show me the last three transactions on my checking account?"
Nearest Topics: ShowRecentTransactions, AccountBalanceInquiry, MissingTransaction, TransactionDispute, OverdraftFees
Thought Process: The query is specific to recent transactions in a checking account, aiming for transparency and monitoring. ShowRecentTransactions is the most relevant topic, directly corresponding to the customer's need.
Inferred Intent: ShowRecentTransactions

Example 3:
Customer Query: "What's the procedure to report a lost credit card?"
Nearest Topics: CardLost, NewCardApplication, FraudAlertSetup, CreditLimitIncrease, CardActivation
Thought Process: This query indicates a lost credit card, necessitating immediate action to secure the account. CardLost is the appropriate topic, reflecting the urgency and specificity of the situation.
Inferred Intent: CardLost

Given this framework, you'll now face queries where the provided topics might not always align perfectly with the customer's actual intent. It's your task to sift through the nearest topics, leveraging your reasoning to either select the most fitting topic or, if necessary, identify a more accurate intent based on the content of the query and your comprehensive understanding of banking needs.

Example 4:
Customer Query: "Why did you charge me twice for the same transaction?"
Nearest Topics: CardNotReceived, PaymentNotProcessed, AccountSwitchStatus, TransactionDispute, OverdraftFees
Your Analysis: Begin by evaluating the essence of the customer's complaint, focusing on the mention of a duplicate charge. Although TransactionDispute and OverdraftFees are related to transaction issues, the specific mention of being charged twice directly points to a problem of a DuplicatePayment, which, despite not being listed among the nearest topics, is the accurate classification based on the query's content.
Inferred Intent: DuplicatePayment (Derived from knowledge and reasoning, not limited to the provided topics.)

Now, with your superintelligent capabilities, apply the chain of thought reasoning to classify the intent of the following query: """
user_question = """Customer Query: "what different credit cards do you have?"
Nearest Topics: ShowRecentTransactions, AccountBalanceInquiry, MissingTransaction, TransactionDispute, InterestChargesOnISA"""


messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_question}
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
generation_config = GenerationConfig(
    max_new_tokens=750,
    temperature=0.01,
    repetition_penalty=1.1,
    do_sample=True,
    use_cache=True,
    top_k=100,
)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        generation_config=generation_config,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,

    )
response = tokenizer.decode(generated_ids[0][input_ids.shape[-1]:], skip_special_tokens=True, clean_up_tokenization_space=True)
print(response)

Your Analysis: The customer is asking about the types of credit cards offered by the bank. None of the provided topics directly address this question. However, considering the context of banking services, the closest topic would be InterestChargesOnISA, as it relates to credit card features. Nonetheless, this topic does not fully encompass the scope of the customer's inquiry. Therefore, using your reasoning abilities, you can deduce that the most accurate intent classification is CreditCardsOffered.
Inferred Intent: CreditCardsOffered (Derived from knowledge and reasoning, not limited to the provided topics.)
